In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib.pyplot as plt

import tfz
from utils import plot_mfs

%matplotlib inline
plt.style.use('seaborn-deep')
plt.rcParams['figure.figsize'] = (10.5, 6)

In [3]:
SUBJECT = 'all'
ARCHITECTURE = '3-3-2-2-2-3-3'

arch_file = 'fcs-description-{}-{}.csv'.format(SUBJECT, ARCHITECTURE)
outp_file = 'fcs-outputs-{}-{}.csv'.format(SUBJECT, ARCHITECTURE)

In [4]:
VARIABLES = (
    'Leader distance', 'Next TLS distance', 'Next TLS green',
    'Next TLS yellow', 'Next TLS red', 'Speed', 'Speed to leader',
)
VARS = [''.join(s[:1].upper() + s[1:] for s in i.split(' ')) for i in VARIABLES]

## Mejor arquitectura

### Particiones

In [5]:
row = pd.read_csv(arch_file).iloc[0]
variables_values = []
for variable in VARS:
    keys = [c for c in row.keys() if c.startswith(variable)]
    variables_values.append({
        k.split('/')[-1]: row[k] for k in keys
    })

In [6]:
row

LeaderDistance/b                                                        0
LeaderDistance/s0                                                     0.2
LeaderDistance/s1                                                     0.2
LeaderDistance/s2                                                     0.2
LeaderDistance/s3                                                     0.2
NextTLSDistance/b                                                       0
NextTLSDistance/s0                                                    0.2
NextTLSDistance/s1                                                    0.2
NextTLSDistance/s2                                                    0.2
NextTLSDistance/s3                                                    0.2
NextTLSGreen/b                                                          0
NextTLSGreen/s0                                                  0.333333
NextTLSGreen/s1                                                  0.333333
NextTLSRed/b                          

In [15]:
for i, (name, desc) in enumerate(zip(VARS, variables_values)):
    tf.reset_default_graph()

    for k, v in desc.items():
        desc[k] = np.float32(v)
    
    # Input data
    x = tf.placeholder(dtype=tf.float32)
    
    # Variables
    base = tf.constant(desc['b'], dtype=tf.float32)
    next_fs_starting_point = base + desc['s0']
    
    num_fs = ((len(desc) - 1) // 2) + 1
    mfs = []
    for i in range(num_fs):
        if i == 0:
            mf = tfz.slope_desc(x, next_fs_starting_point, desc['s1'])
        elif i == num_fs - 1:
            mf = tfz.slope_asc(x, next_fs_starting_point, desc['s1'])
        else:
            sta = ['s' + str(i) for i in range(i * 2 - 1, i * 2 + 2)]
            mf = tfz.trapezoid(x, next_fs_starting_point, sta[0], sta[1], sta[2])
            next_fs_starting_point += shifts_to_apply[1] + shifts_to_apply[2]

        mfs.append(mf)

    min_x, max_x = desc['b1'], desc['s0'] * num_fs
    X = np.reshape(np.linspace(min_x, max_x, 200), (-1, 1))
    print(type(X))

    with tf.Session() as session:
        fig, ax = plt.subplots(1, 1)
        ax.set_title(name)
        ax.set_xlim(min_x, max_x)
        ax.set_ylim(0, 1)

        for Y in session.run(mfs, feed_dict={x: X}):
            ax.plot(X, Y)

TypeError: Value passed to parameter 'x' has DataType string not in list of allowed values: float16, bfloat16, float32, float64, int32, int64